In [79]:
import pandas as pd
import numpy as np

In [80]:
data1 = pd.read_csv('nyctastematch-restaurant-recommender/data_cleaning/restaurants_data_cleaned.csv')
data2 = pd.read_csv('nyctastematch-restaurant-recommender/data_cleaning/cleaned_tripadvisor_ny_restaurants.csv')
data3 = pd.read_csv('nyctastematch-restaurant-recommender/data_cleaning/nyc_restaurant_week_2018_cleaned.csv')

In [81]:
data1.head()

,name,price,rating,review_count,cuisine,categories,address,display_phone
0,123 Burger Shot Beer,1,3.0,1000,American,"['sportsbars', 'tradamerican', 'chicken_wings']","738 10th Ave, Hells Kitchen, NY 10019",(212) 315-0123
1,One Stop Patty Shop,1,4.0,40,Bakery,"['caribbean', 'breakfast_brunch']","1708 Amsterdam Ave, Harlem, NY 10031",(212) 491-7466
2,108 Food Dried Hot Pot,2,3.5,139,Chinese,"['hotpot', 'chinese']","2794 Broadway, East Harlem, NY 10025",(917) 675-6878
3,Cookshop,2,4.0,1000,American,"['newamerican', 'breakfast_brunch', 'wine_bars']","156 10th Ave, Midtown West, NY 10011",(212) 924-4440
4,11 Hanover Greek,3,4.0,122,Greek,"['greek', 'seafood', 'wine_bars']","11 Hanover Sq, Tribeca, NY 10005",(212) 785-4000


In [82]:
data2.head()

,Name,Type,Rating,No of Reviews,Comments,Contact Number,Price,Address
0,Table Talk Diner,"American, Diner, Vegetarian Friendly",4.0,256.0,Waitress was very friendly but a little pricey...,+1 845-849-2839,2.5,"2521 South Rd Ste C, Poughkeepsie, NY 12601-5476"
1,The Clam Bar,"American, Bar, Seafood",4.0,285.0,Doesn't look like much from the outside but wa...,+1 315-458-1662,2.5,"3914 Brewerton Rd, Syracuse, NY 13212"
2,John Thomas Steakhouse,"American, Steakhouse, Gluten Free Options",4.0,258.0,"The ambience, food and service were all excell...",+1 607-273-3464,4.0,"1152 Danby Rd, Ithaca, NY 14850-8927"
3,ZaZa's Cucina,"Italian, Vegetarian Friendly, Gluten Free Opt...",4.0,271.0,"Nice big dining area. Holds many people, so n...",+1 607-273-9292,2.5,"622 Cascadilla St, Ithaca, NY 14850-4049"
4,New World Bistro Bar,"American, Bar, Vegetarian Friendly",4.5,706.0,"Came with a group of eight people, and had a w...",+1 518-694-0520,2.5,"300 Delaware Ave, Albany, NY 12209-1627"


In [83]:
data3.head()

,Unnamed: 0,name,average_review,review_count,price_range,street_address,restaurant_type
0,0,Smoke Jazz and Supper Club,4.421550,2155,$31 to $50,"2751 Broadway New York, NY 10025",Contemporary American
1,1,Tavern on the Green,4.670290,7029,$31 to $50,"1 Tavern on the Green New York, NY 10023",American
2,2,ABC Kitchen,4.760310,6031,$31 to $50,"35 East 18th Street New York, NY 10003",Contemporary American
3,3,Catch New York,4.548300,4830,$31 to $50,"21 Ninth Avenue New York, NY 10014",Seafood
4,4,Becco,4.418139,18139,$30 and under,"355 West 46th Street New York, NY 10010",Italian


In [84]:
print(f"Dataset 1: {data1.shape}")
print(f"Dataset 2: {data2.shape}")
print(f"Dataset 3: {data3.shape}")

Dataset 1: (3913, 8)
Dataset 2: (593, 8)
Dataset 3: (348, 7)


In [85]:
# Standardize column names across all datasets
data3 = data3.rename(columns={
    'average_rating': 'rating',
    'review_count': 'review_count',
    'price_range': 'price',
    'street_address': 'address',
    'restaurant_type': 'cuisine'
})

# Dataset 3 - rename columns
data2 = data2.rename(columns={
    'Name': 'name',
    'Type': 'cuisine',  
    'Rating': 'rating',
    'No of Reviews': 'review_count',
    'Contact Number': 'display_phone',
    'Price': 'price',
    'Address': 'address',
    'Comments': 'comments'
})

In [86]:
import ast

# ============================================
# Dataset 1: Merge cuisine and categories_clean
# ============================================
def merge_cuisine_categories_data1(row):
    cuisines = []
    
    # Add cuisine if exists
    if pd.notna(row['cuisine']):
        cuisines.append(str(row['cuisine']).lower().strip().replace(' ', '_'))
    
    # Add categories if exists
    if pd.notna(row['categories']):
        if isinstance(row['categories'], str):
            # Parse string representation of list
            try:
                cats = ast.literal_eval(row['categories'])
            except:
                cats = []
        elif isinstance(row['categories'], list):
            cats = row['categories']
        else:
            cats = []
        
        # Add each category
        for cat in cats:
            cat_clean = str(cat).lower().strip().replace(' ', '_')
            if cat_clean and cat_clean not in cuisines:
                cuisines.append(cat_clean)
    
    return cuisines if cuisines else []

data1['cuisine'] = data1.apply(merge_cuisine_categories_data1, axis=1)

# Drop the old categories column
if 'categories' in data1.columns:
    data1 = data1.drop(columns=['categories'])
if 'categories_clean' in data1.columns:
    data1 = data1.drop(columns=['categories_clean'])

print("Dataset 1 after merging:")
print(data1[['name', 'cuisine']].head())


# ============================================
# Dataset 2: Convert cuisine to list (already renamed from Type)
# ============================================
def parse_cuisine_data2(type_str):
    if pd.isna(type_str):
        return []
    
    # Clean and convert to list
    cuisines = [c.strip().lower().replace(' ', '_') for c in str(type_str).split(',')]
    
    # Remove duplicates while preserving order
    seen = set()
    unique_cuisines = []
    for cuisine in cuisines:
        if cuisine and cuisine not in seen:
            seen.add(cuisine)
            unique_cuisines.append(cuisine)
    
    return unique_cuisines

data2['cuisine'] = data2['cuisine'].apply(parse_cuisine_data2)  # Changed from restaurant_type to cuisine

print("\nDataset 2 after processing:")
print(data2[['name', 'cuisine']].head())


# ============================================
# Dataset 3: Convert cuisine to list (already renamed from restaurant_type)
# ============================================
def parse_cuisine_data3(type_str):
    if pd.isna(type_str):
        return []
    
    # Split by comma and clean
    cuisines = [c.strip().lower().replace(' ', '_') for c in str(type_str).split(',')]
    
    # Remove duplicates while preserving order
    seen = set()
    unique_cuisines = []
    for cuisine in cuisines:
        if cuisine and cuisine not in seen:
            seen.add(cuisine)
            unique_cuisines.append(cuisine)
    
    return unique_cuisines

data3['cuisine'] = data3['cuisine'].apply(parse_cuisine_data3)  # Changed from Type to cuisine

print("\nDataset 3 after processing:")
print(data3[['name', 'cuisine']].head())

Dataset 1 after merging:
                     name                                            cuisine
0    123 Burger Shot Beer  [american, sportsbars, tradamerican, chicken_w...
1     One Stop Patty Shop              [bakery, caribbean, breakfast_brunch]
2  108 Food Dried Hot Pot                                  [chinese, hotpot]
3                Cookshop  [american, newamerican, breakfast_brunch, wine...
4        11 Hanover Greek                        [greek, seafood, wine_bars]

Dataset 2 after processing:
                     name                                            cuisine
0        Table Talk Diner             [american, diner, vegetarian_friendly]
1            The Clam Bar                           [american, bar, seafood]
2  John Thomas Steakhouse        [american, steakhouse, gluten_free_options]
3           ZaZa's Cucina  [italian, vegetarian_friendly, gluten_free_opt...
4    New World Bistro Bar               [american, bar, vegetarian_friendly]

Dataset 3 after proce

In [87]:
# Dataset 1: Already has price as 1, 2, 3, 4
# (assuming you already converted it)

# Dataset 2: Convert price_range ($31 to $50) to numeric
def parse_price_range_dataset2(price_str):
    if pd.isna(price_str):
        return None
    
    price_str = str(price_str).lower()
    
    # Extract numbers from strings like "$31 to $50"
    if 'to' in price_str:
        # Get the upper bound
        upper = price_str.split('to')[1].strip().replace('$', '').strip()
        try:
            upper_value = int(upper)
            if upper_value <= 15:
                return 1
            elif upper_value <= 30:
                return 2
            elif upper_value <= 60:
                return 3
            else:
                return 4
        except:
            return None
    
    return None

data2['price'] = data2['price'].apply(parse_price_range_dataset2)

# Dataset 3: Convert price (2.5, 4, etc.) to 1-4 scale
def parse_price_dataset3(price_val):
    if pd.isna(price_val):
        return None
    
    try:
        price_float = float(price_val)
        # Assuming scale is 1-5, convert to 1-4
        if price_float <= 1.5:
            return 1
        elif price_float <= 2.5:
            return 2
        elif price_float <= 3.5:
            return 3
        else:
            return 4
    except:
        return None

data3['price'] = data3['price'].apply(parse_price_dataset3)

In [88]:
# Define the standard columns you want in the final dataset
standard_columns = [
    'name',
    'price',
    'rating',
    'review_count',
    'cuisine',
    'categories',
    'address',
    'display_phone',
    'comments'  # Only from Dataset 3, will be None for others
]

# Add missing columns with None/NaN values
for data in [data1, data2, data3]:
    for col in standard_columns:
        if col not in data.columns:
            data[col] = None

# Select only standard columns from each dataset
data1_standard = data1[standard_columns].copy()
data2_standard = data2[standard_columns].copy()
data3_standard = data3[standard_columns].copy()

# Merge all datasets
data_merged = pd.concat([data1_standard, data2_standard, data3_standard], ignore_index=True)

data_merged = data_merged.drop(columns=['categories'])

print(f"\nMerged dataset shape: {data_merged.shape}")
print(f"Columns: {list(data_merged.columns)}")


Merged dataset shape: (4854, 8)
Columns: ['name', 'price', 'rating', 'review_count', 'cuisine', 'address', 'display_phone', 'comments']


C:\Users\shrep\AppData\Local\Temp\ipykernel_14872\983958871.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_merged = pd.concat([data1_standard, data2_standard, data3_standard], ignore_index=True)


In [89]:
# Remove duplicates based on name and address
data_merged = data_merged.drop_duplicates(subset=['name', 'address'], keep='first')

# Clean up
data_merged = data_merged.reset_index(drop=True)

# Check for missing values
print("\nMissing values per column:")
print(data_merged.isnull().sum())

print(f"\nFinal dataset shape after deduplication: {data_merged.shape}")


Missing values per column:
name                0
price             870
rating            348
review_count        0
cuisine             0
address             0
display_phone     348
comments         4076
dtype: int64

Final dataset shape after deduplication: (4598, 8)


In [91]:
import numpy as np

# ============================================
# 1. Fill missing phone numbers and comments with 'N/A'
# ============================================
data_merged['display_phone'] = data_merged['display_phone'].fillna('N/A')
data_merged['comments'] = data_merged['comments'].fillna('N/A')

print("Phone and comments filled with N/A")


# ============================================
# 2. Fill missing ratings from data3's average_rating
# ============================================
# First, load data3 again to get average_rating (or if you still have it)
# Assuming you still have data3 with average_rating column

# Create a mapping from data3: name -> average_rating (rounded)
data3_rating_map = data3.set_index('name')['average_review'].to_dict()

# Function to fill missing ratings
def fill_rating(row):
    if pd.isna(row['rating']) and row['name'] in data3_rating_map:
        return round(data3_rating_map[row['name']])
    return row['rating']

data_merged['rating'] = data_merged.apply(fill_rating, axis=1)

print(f"\nRatings filled. Remaining null ratings: {data_merged['rating'].isna().sum()}")


# ============================================
# 3. Fill missing prices from data3's price_range
# ============================================
# Create a mapping from data3: name -> price (converted from price_range)
def convert_price_range_data3(price_range):
    if pd.isna(price_range):
        return None
    
    price_str = str(price_range).strip()
    
    # Check if it's "$31 to $50" format
    if "$31 to $50" in price_str or "$31-$50" in price_str:
        return 2
    else:
        return 1

# Apply to data3 if it still has price_range column
if 'price_range' in data3.columns:
    data3_price_map = {}
    for idx, row in data3.iterrows():
        data3_price_map[row['name']] = convert_price_range_data3(row.get('price_range'))

    # Fill missing prices in merged dataset
    def fill_price_from_data3(row):
        if pd.isna(row['price']) and row['name'] in data3_price_map:
            return data3_price_map[row['name']]
        return row['price']

    data_merged['price'] = data_merged.apply(fill_price_from_data3, axis=1)

print(f"Prices from data3 filled. Remaining null prices: {data_merged['price'].isna().sum()}")


# ============================================
# 4. Fill missing prices from data2's Price column
# ============================================
# Create a mapping from data2: name -> price (converted from Price text)
def convert_price_data2(price_val):
    if pd.isna(price_val):
        return None
    
    try:
        # If it's already numeric (like 2.5, 4), convert to 1-4 scale
        price_float = float(price_val)
        if price_float <= 1.5:
            return 1
        elif price_float <= 2.5:
            return 2
        elif price_float <= 3.5:
            return 3
        else:
            return 4
    except:
        # If it's text like "$$ - $$$", parse it
        price_str = str(price_val).lower()
        dollar_count = price_str.count('$')
        
        if dollar_count == 0:
            return None
        elif dollar_count <= 2:
            return 1 if dollar_count == 1 else 2
        elif dollar_count <= 6:
            return 3
        else:
            return 4

# Apply to data2 if it still has Price column
if 'Price' in data2.columns:
    data2_price_map = {}
    for idx, row in data2.iterrows():
        data2_price_map[row['name']] = convert_price_data2(row.get('Price'))

    # Fill missing prices in merged dataset
    def fill_price_from_data2(row):
        if pd.isna(row['price']) and row['name'] in data2_price_map:
            return data2_price_map[row['name']]
        return row['price']

    data_merged['price'] = data_merged.apply(fill_price_from_data2, axis=1)

print(f"Prices from data2 filled. Remaining null prices: {data_merged['price'].isna().sum()}")


# ============================================
# Convert price to int
# ============================================
data_merged['price'] = data_merged['price'].fillna(0).astype(int)

# Replace 0 with the mode (most common price)
if (data_merged['price'] == 0).sum() > 0:
    price_mode = data_merged[data_merged['price'] > 0]['price'].mode()[0]
    data_merged['price'] = data_merged['price'].replace(0, price_mode)

print(f"\nPrice conversion complete. Price distribution:")
print(data_merged['price'].value_counts().sort_index())


# ============================================
# Final verification
# ============================================
print("\n=== Final Missing Values ===")
print(data_merged.isnull().sum())

print("\n=== Data Types ===")
print(data_merged.dtypes)

print("\n=== Sample Rows ===")
print(data_merged.head())

Phone and comments filled with N/A

Ratings filled. Remaining null ratings: 0
Prices from data3 filled. Remaining null prices: 870
Prices from data2 filled. Remaining null prices: 870

Price conversion complete. Price distribution:
price
1     621
2    3413
3     438
4     126
Name: count, dtype: int64

=== Final Missing Values ===
name             0
price            0
rating           0
review_count     0
cuisine          0
address          0
display_phone    0
comments         0
dtype: int64

=== Data Types ===
name              object
price              int64
rating           float64
review_count     float64
cuisine           object
address           object
display_phone     object
comments          object
dtype: object

=== Sample Rows ===
                     name  price  rating  review_count  \
0    123 Burger Shot Beer      1     3.0        1000.0   
1     One Stop Patty Shop      1     4.0          40.0   
2  108 Food Dried Hot Pot      2     3.5         139.0   
3             

C:\Users\shrep\AppData\Local\Temp\ipykernel_14872\1070772587.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_merged['price'] = data_merged['price'].fillna(0).astype(int)


In [ ]:

# Save the merged dataset
data_merged.to_csv('restaurants_data_merged_final.csv', index=False)

print(f"\nMerged dataset saved successfully!")
print(f"Total restaurants: {len(data_merged)}")
print(f"\nSample row:")
print(data_merged.iloc[0])


Merged dataset saved successfully!
Total restaurants: 4598

Sample row:
name                                          123 Burger Shot Beer
price                                                            1
rating                                                         3.0
review_count                                                1000.0
cuisine          [american, sportsbars, tradamerican, chicken_w...
address                      738 10th Ave, Hells Kitchen, NY 10019
display_phone                                       (212) 315-0123
comments                                                       N/A
Name: 0, dtype: object
